In [115]:
!pip install seaborn, scipy

ERROR: Invalid requirement: 'seaborn,'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.stats import pearsonr
import ast
# Load the dataset
data = pd.read_csv('book_dataset.csv')
columns_to_keep = ['title', 'author_name', 'ratings_average', 'ratings_count', 'subject','first_publish_year','key','language','readinglog_count',]
data = data[columns_to_keep]
data.tail(20)

C:\Users\VIDHI\AppData\Local\Temp\ipykernel_17932\230437597.py:10: DtypeWarning: Columns (45,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('book_dataset.csv')


,title,author_name,ratings_average,ratings_count,subject,first_publish_year,key,language,readinglog_count
9980,Learn Cocoa on the Mac,['Jack Nutting'],NaN,NaN,"['Mac OS', 'Application program interfaces (Co...",2011.0,/works/OL19548291W,['eng'],NaN
9981,Experimental Research in Evolutionary Computation,['Thomas Bartz-Beielstein'],NaN,NaN,['Evolutionary programming (Computer science)'...,2006.0,/works/OL9032175W,['eng'],0.0
9982,Unified software engineering with Java,"['Georges G. Merx', 'Ronald J. Norman']",NaN,NaN,"['Java (Computer program language)', 'Object-o...",2006.0,/works/OL17837699W,['eng'],NaN
9983,Handbook of Visual Display Technology,['Janglin Chen'],NaN,NaN,"['Computer Imaging, Vision, Pattern Recognitio...",2012.0,/works/OL19854591W,['eng'],NaN
9984,"MARINE 2011, IV International Conference on Co...",['Luís Eça'],NaN,NaN,"['Offshore Engineering', 'Computational Mathem...",2013.0,/works/OL19885728W,['eng'],NaN
9985,Multi-objective optimization using evolutionar...,['Kalyanmoy Deb'],NaN,NaN,['Evolutionary programming (Computer science)'...,2001.0,/works/OL8244342W,['eng'],NaN
9986,Computational neural networks for geophysical ...,['M.M. Poulton'],NaN,NaN,"['Geophysical methods', 'Neural networks (Comp...",2001.0,/works/OL16976910W,['eng'],0.0
9987,Proceedings,"['TENCON (1993 Beijing, China)', 'China) Trend...",NaN,NaN,"['Automatic control', 'Computer networks', 'Co...",1993.0,/works/OL3969132W,['eng'],0.0
9988,Logic Programming with Prolog,['Max Bramer'],NaN,NaN,"['Logic programming', 'Prolog (computer progra...",2005.0,/works/OL25619085W,['eng'],0.0
9989,Elements of computer science,"['J. Jha', 'P. K. Mahanti', 'L. Sahoo']",NaN,NaN,"['Computer science', 'General Theory of Comput...",1996.0,/works/OL19031513W,['eng'],NaN


## 1)Dataset Overview
Let's check for missing values and get a basic understanding of the data distribution.

In [117]:

data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               10000 non-null  object 
 1   author_name         9967 non-null   object 
 2   ratings_average     719 non-null    float64
 3   ratings_count       719 non-null    float64
 4   subject             10000 non-null  object 
 5   first_publish_year  9991 non-null   float64
 6   key                 10000 non-null  object 
 7   language            9559 non-null   object 
 8   readinglog_count    6693 non-null   float64
dtypes: float64(4), object(5)
memory usage: 703.3+ KB


title                    0
author_name             33
ratings_average       9281
ratings_count         9281
subject                  0
first_publish_year       9
key                      0
language               441
readinglog_count      3307
dtype: int64

## 2) Data Preprocessing and feature engineering
We'll clean the data, fill in or drop missing values, and ensure correct data types for analysis.

In [118]:
# Drop rows with missing essential values in title or author_name columns
data.dropna(subset=['title', 'author_name'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9967 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               9967 non-null   object 
 1   author_name         9967 non-null   object 
 2   ratings_average     717 non-null    float64
 3   ratings_count       717 non-null    float64
 4   subject             9967 non-null   object 
 5   first_publish_year  9960 non-null   float64
 6   key                 9967 non-null   object 
 7   language            9528 non-null   object 
 8   readinglog_count    6667 non-null   float64
dtypes: float64(4), object(5)
memory usage: 778.7+ KB


Handling missing values for Different Features


In [119]:
# Missing Value Count Function
def show_missing(y):
    missing = y.columns[y.isnull().any()].tolist()
    return missing

def print_missing(x):
  # Missing data counts and percentage
  print('Missing Data Count')
  print(x[show_missing(x)].isnull().sum().sort_values(ascending = False))
  print('--'*50)
  print('Missing Data Percentage')
  print(round(x[show_missing(x)].isnull().sum().sort_values(ascending = False)/len(x)*100,2))

In [120]:
print_missing(data)

Missing Data Count
ratings_average       9250
ratings_count         9250
readinglog_count      3300
language               439
first_publish_year       7
dtype: int64
----------------------------------------------------------------------------------------------------
Missing Data Percentage
ratings_average       92.81
ratings_count         92.81
readinglog_count      33.11
language               4.40
first_publish_year     0.07
dtype: float64


#### Handling the missing values of readinglog_count

In [121]:
# Null values in readinglog_count column
nulls = sum(data['readinglog_count'].isnull())
print('Null values in readinglog_count: ',nulls)

Null values in readinglog_count:  3300


In [122]:
# Replacing null values
median = data['readinglog_count'].median()
std = data['readinglog_count'].std()
random_readingCount = np.random.randint(median - std, median + std, size = nulls)
readingCount = data['readinglog_count'].copy()
readingCount[pd.isnull(readingCount)] = random_readingCount
data['readinglog_count'] = readingCount
data['readinglog_count'] = data['readinglog_count'].astype(int)

# 3) Building Models

## 1) Top 10 books for respective authors

## 2) Content based filtering

In [123]:
!pip install nltk
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
import string


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [124]:
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VIDHI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\VIDHI\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [125]:
stopwords.words('english').extend(['\n','\t'])

In [126]:
#combined feature string
data['combined_features']=data['title']+' '+data['author_name']+' '+data['subject'].astype(str)
data[['title','combined_features']].head()

,title,combined_features
0,2001,2001 ['Arthur C. Clarke'] ['Human-computer int...
1,"I, Robot","I, Robot ['Isaac Asimov'] ['smear campaigns', ..."
2,Prey,"Prey ['Michael Crichton'] ['programmers', 'nan..."
3,Neuromancer,Neuromancer ['William Gibson'] ['Information s...
4,Artemis Fowl and the Eternity Code,Artemis Fowl and the Eternity Code ['Eoin Colf...


In [127]:
#Removing Stop Words and Punctuations
def text_process(msg):
    nopunc =[char for char in msg if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])
data['title']=data['title'].apply(text_process)

In [128]:
nltk.download('wordnet')
# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()

def lemma(text):    
    '''a function which stems each word in the given text'''
    text = [wnl.lemmatize(word) for word in text.split()]
    return " ".join(text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VIDHI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [129]:
data['title']=data['title'].apply(lemma)

#### Vectorizing the data

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [131]:
# Vectorize the combined features
tfidf = TfidfVectorizer(min_df=5)
tfidf_matrix = tfidf.fit_transform(data['combined_features'])
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 175352 stored elements and shape (9967, 3646)>

In [132]:
# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert cosine similarity to a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['title'], columns=data['title'])

#### Recommendation Function

1) Author based Similarity

In [133]:
def recommend_by_author(title, data, top_n=10):
    # Find the author of the input book
    input_author = data[data['title'] == title]['author_name'].values[0]
    
    # Filter books by the same author
    same_author_books = data[data['author_name'] == input_author]
    
    # Return the top N books by the same author, excluding the input book itself
    recommendations = same_author_books[same_author_books['title'] != title].head(top_n)
    return recommendations[['title', 'author_name', 'subject']]


2) Genre based similarity
Using Jaccard similarity on genres using a binary encoding.

In [134]:
def recommend_by_genre(title, data, top_n=10):
    # Find the genres of the input book
    input_genres = set(data[data['title'] == title]['subject'].values[0].split(', '))
    
    # Calculate Jaccard similarity based on genre overlap
    def genre_similarity(book_genres):
        book_genres = set(book_genres.split(', '))
        return len(input_genres.intersection(book_genres)) / len(input_genres.union(book_genres))
    
    # Apply genre similarity and filter top N matches
    data['genre_similarity'] = data['subject'].apply(genre_similarity)
    top_genre_matches = data.sort_values(by='genre_similarity', ascending=False)
    recommendations = top_genre_matches[top_genre_matches['title'] != title].head(top_n)
    
    return recommendations[['title', 'author_name', 'subject', 'genre_similarity']]

3) Overall similarity recommendations 
Using combined TF-IDF vectorizer

In [137]:
def recommend_books(title, cosine_sim_df, data, top_n=10):
    if title not in cosine_sim_df.index:
        return f"Book '{title}' not found in the dataset."
    
    sim_scores = cosine_sim_df[title]
    similar_books = sim_scores.sort_values(ascending=False).head(top_n + 1)  # +1 to exclude the book itself
    recommended_books = data[data['title'].isin(similar_books.index[1:])]

    return recommended_books[['title', 'author_name', 'subject']]

In [148]:
title = input("Enter the book title: ")
if title:
    recommended_books = recommend_books(title, pd.DataFrame(cosine_sim, index=data['title'], columns=data['title']), data)
    
    # Check if the returned value is an error message
    if isinstance(recommended_books, str):
        print("Error:", recommended_books)  # Display error if book not found
    else:
        print("\nTop Recommendations Based on Author Similarity:")
        author_recommendations = recommend_by_author(title, data)
        display(author_recommendations)

        print("\nTop Recommendations Based on Genre Similarity:")
        genre_recommendations = recommend_by_genre(title, data)
        display(genre_recommendations)
        
        print("\ntop 10 Recommended Books based on overall similarity:")
        overall_recommendations = recommend_books(title, pd.DataFrame(cosine_sim, index=data['title'], columns=data['title']), data)
        display(overall_recommendations)
else:
    print("Please enter a book title.")


Top Recommendations Based on Author Similarity:


,title,author_name,subject
1687,Zendegi,['Greg Egan'],"['Brain', 'Virtual reality', 'Computer simulat..."



Top Recommendations Based on Genre Similarity:


,title,author_name,subject,genre_similarity
2548,pioneer,['Frederik Pohl'],"['Computer hackers', 'Fiction', 'Science ficti...",0.384615
659,Saturns race,"['Larry Niven', 'Steven Barnes']","['Conspiracies', 'Utopias', 'Computers', 'Fict...",0.357143
668,Spaceland,['Rudy Rucker'],"['Computer industry', 'Fourth dimension', 'Fic...",0.333333
967,Better Life Red Dwarf,['Grant Naylor'],"['Psychological games', 'Computers', 'Fiction'...",0.333333
4186,Lost Tribes,['C. Taylor-Butler'],"['Computer games', 'Science fiction', 'Riddles...",0.312500
7430,maelstrom,['Loren L. Coleman'],"['American Science fiction', 'Fiction', 'Scien...",0.307692
1419,Archangel protocol,['Lyda Morehouse'],"['Cyberspace', 'Computers', 'Angels', 'Fiction...",0.307692
6166,MagicNet,['John DeChancie'],"['Computer networks', 'Fiction', 'Fiction, sci...",0.307692
2569,labyrinth key,['Howard V. Hendrix'],"['Computer engineering', 'Data encryption (Com...",0.285714
2547,Accel World,['Reki Kawahara'],"['Bullying', 'Virtual reality', 'Japanese Scie...",0.285714



top 10 Recommended Books based on overall similarity:


,title,author_name,subject
16,Stand Zanzibar,['John Brunner'],"['Fiction in English', 'Science fiction', 'Hug..."
303,Rhesus Chart,['Charles Stross'],"['Fiction, thrillers, general', 'Fiction, susp..."
659,Saturns race,"['Larry Niven', 'Steven Barnes']","['Conspiracies', 'Utopias', 'Computers', 'Fict..."
720,Awakening Orbis 4,['PJ Haarsma'],"['Pirates', 'Orphans', 'Computers', 'Space and..."
1419,Archangel protocol,['Lyda Morehouse'],"['Cyberspace', 'Computers', 'Angels', 'Fiction..."
2000,Human4,['Mike A. Lancaster'],"[""Children's fiction"", 'Science fiction', 'Com..."
2547,Accel World,['Reki Kawahara'],"['Bullying', 'Virtual reality', 'Japanese Scie..."
6166,MagicNet,['John DeChancie'],"['Computer networks', 'Fiction', 'Fiction, sci..."
6912,Virus,['Graham Watkins'],"['Computer viruses', 'Physicians', 'Fiction', ..."
7430,maelstrom,['Loren L. Coleman'],"['American Science fiction', 'Fiction', 'Scien..."


### KNN Similarity

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score, recall_score, f1_score
import random


In [142]:
data['combined'] = data['title'] + ' ' + data['author_name'] + ' ' + data['subject'].astype(str)
data.dropna(subset=['combined'], inplace=True)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['combined'])

In [143]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(X)
n_clusters = 10  # You can adjust this
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=10, random_state=42)

In [144]:
def get_recommendations(book_title):
    if book_title not in data['title'].values:
        return "Book not found in the dataset."
    
    idx = data[data['title'] == book_title].index[0]
    distances, indices = knn.kneighbors(X[idx], n_neighbors=10)
    recommended_books = data.iloc[indices[0]]['title'].values
    return recommended_books

In [145]:
def evaluate_relevance(book_title):
    recommended_books = get_recommendations(book_title)

    if isinstance(recommended_books, str):  # Check if the book was not found
        return []

    # Get the cluster of the input book
    book_idx = data[data['title'] == book_title].index[0]
    book_cluster = kmeans.labels_[book_idx]

    # Calculate relevance based on clustering
    relevant_books = []
    for recommended_book in recommended_books:
        idx = data[data['title'] == recommended_book].index[0]
        if kmeans.labels_[idx] == book_cluster:
            relevant_books.append(recommended_book)
    
    return relevant_books

In [146]:
def evaluate_model(true_relevant_books,recommendations):
    relevant_books = set(true_relevant_books)
    hits = [book for book in recommendations if book in relevant_books]
    hits_count = len(hits)
    
    recommended_count = len(recommendations)
    top_n=10
    
    recall = hits_count/min(top_n,len(relevant_books))
    precision = hits_count/top_n
    accuracy = hits_count/recommended_count if recommended_count > 0 else 0
    
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0  # avoid division by zero
    
    print(f"Recall @ Top {top_n}: {recall:.4f}")
    print(f"Precision @ Top {top_n}: {precision:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [147]:
title = input("Enter the book title: ")
if title:
    recommended_books = get_recommendations(title)
    
    # Check if the returned value is an error message
    if isinstance(recommended_books, str):
        print("Error:", recommended_books)  # Display error if book not found
    else:
        print("\nTop 10 Recommended Books based on KNN similarity:")
        print(f"{recommended_books}")
else:
    print("Please enter a book title.")
print("Evaluation when K-means clustering is used to find relevant books")
print (evaluate_relevance(title))
evaluate_model(evaluate_relevance(title),recommended_books)
print("Evaluation when content-based filtering is used to find relevant books")
evaluate_model(recommend_books(title, pd.DataFrame(cosine_sim, index=data['title'], columns=data['title']), data)['title'],recommended_books)


Top 10 Recommended Books based on KNN similarity:
['Snow Crash' 'Rhesus Chart' 'Diaspora' 'Human4' 'Diggers' 'maelstrom'
 'Code' 'Cryptonomicon' 'Halting State' 'Virus']
Evaluation when K-means clustering is used to find relevant books
['Snow Crash', 'Rhesus Chart', 'Diaspora', 'Human4', 'Diggers', 'Cryptonomicon', 'Halting State']
Recall @ Top 10: 1.0000
Precision @ Top 10: 0.7000
Accuracy: 0.7000
F1 Score: 0.8235
Evaluation when content-based filtering is used to find relevant books
Recall @ Top 10: 0.6000
Precision @ Top 10: 0.6000
Accuracy: 0.6000
F1 Score: 0.6000
